In [1]:
import numpy as np
import pandas as pd
import sympy

from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, output_file, show
from bokeh.transform import dodge, factor_cmap
# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

Loading BokehJS ...

In [4]:
# create the initial structure for defining things with a dictionary
# the data source will ultimately be a pandas.DataFrame

N = 100 # integers from 1 upto and including N
# use 'unknown' to say we don't know yet if an integer is prime
# use True if it is a prime
# use False if it is not a prime
candidates = {ii: "unknown" for ii in range(1, N + 1)}
candidates[1] = False
f"{len(candidates)=}"

'len(candidates)=100'

In [5]:
# start making the dataframe
df = pd.DataFrame(candidates.items())
df.columns = ["integer", "is_prime"]
df.set_index('integer')
df.head()

,integer,is_prime
0,1,False
1,2,unknown
2,3,unknown
3,4,unknown
4,5,unknown


In [6]:
# Now decorate the dataframe and make it so that it can use strings for most things which is what bokeh needs
# columns = ["I", "II", "III", "IV", "V", "VI", "VII"]
# columns = list(range(1,11))
# rows = [str(x) for x in range(1, 19)]
rows = [str(ii) for ii in range(0, 11)]
columns = [str(ii) for ii in range(10)]


df = pd.DataFrame(candidates.items(), columns=["integer", "is_prime"])
df.set_index('integer')

df["row"] = df.integer.apply(lambda x: (x - 1) // 10) # not sure if these make sense
df["column"] = df.integer.apply(lambda x: (x - 1) % 10)
df["row"] = df["row"].astype(str)
df["column"] = df["column"].astype(str)
df["number"] = df["integer"].astype(str)

In [7]:
print(f"{set(df['row'])=}")
print(f"{set(df['column'])=}")
df.is_prime = df.is_prime.astype(str)
#df.is_prime.to_list()

set(df['row'])={'5', '7', '9', '2', '1', '6', '3', '8', '0', '4'}
set(df['column'])={'5', '7', '9', '2', '1', '6', '3', '8', '0', '4'}


In [8]:
# df["atomic mass"] = df["atomic mass"].astype(str)
# df["row"] = df["row"].astype(str)
# df["column"] = [columns[x-1] for x in df.column]
# df = df[df.row != "-"]
# df = df[df.symbol != "Lr"]
# df = df[df.symbol != "Lu"]

cmap = {
    str(True): "#FFFFFF", # "#a6cee3",
    str(False): "#0000b4",
    # "metal"                : "#d93b43",
    "unknown": "#999d9a",
    # "metalloid"            : "#e08d49",
    # "noble gas"            : "#eaeaea",
    # "nonmetal"             : "#f1d4Af",
    # "transition metal"     : "#599d7A",
}
cmap

{'True': '#FFFFFF', 'False': '#0000b4', 'unknown': '#999d9a'}

In [9]:
TOOLTIPS = [
    ("number", "@number"),
    ("is_prime", "@is_prime")
    # factors?
    #   ("Atomic number", "@{atomic number}"),
    #   ("Atomic mass", "@{atomic mass}"),
    #  ("Type", "@metal"),
    #   ("CPK color", "$color[hex, swatch]:CPK"),
    #  ("Electronic configuration", "@{electronic configuration}"),
]

In [10]:
print(list(cmap.values()))
list(cmap.keys())

['#FFFFFF', '#0000b4', '#999d9a']


['True', 'False', 'unknown']

In [11]:
p = figure(
    title="Primes",
    plot_width=1000,
    plot_height=450,
    x_range=columns,
    y_range=list(reversed(rows)),
    tools="hover",
    toolbar_location=None,
    tooltips=TOOLTIPS,
)

In [12]:


r = p.rect("column", "row", 0.95, 0.95, source=df, fill_alpha=0.6,
    color=factor_cmap('is_prime', palette=list(cmap.values()), factors=list(cmap.keys())))
# legend_field="is_prime",


# color=factor_cmap('unknown', palette=list(cmap.values()), factors=list(cmap.keys())))

In [13]:
#r.data_source.data # this works

In [14]:


text_props = {"source": df, "text_align": "left", "text_baseline": "middle"}
x = dodge("column", -0.4, range=p.x_range)
y = dodge("row", -0.0, range=p.y_range)

p.text(x=x, y=y, text="number", text_font_style="bold", **text_props)


p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
handle = show(p, notebook_handle=True)

def update_is_prime(int_num:int, value: str)-> None:
    df.at[int_num-1,'is_prime'] = str(value)
    r.data_source.data.update(dict(is_prime=df.is_prime.to_numpy()))
    push_notebook(handle=handle)

In [15]:
update_is_prime(2, True)

In [342]:
df.at[3,'is_prime'] = str(True)

In [343]:
df.at[3,'integer']

4

bokeh.models.renderers.GlyphRenderer

In [382]:
#del df.at[100,'is_prime']
df.at[99,'is_prime'] = str(False)

In [383]:
r.data_source.data.update(dict(is_prime=df.is_prime.to_numpy()))
push_notebook(handle=handle)

In [347]:
df.at[10, 'is_prime'] = str(False)
push_notebook(handle=handle)

In [346]:
handle

In [203]:
p.x_range.to_json_string(0)

'{"factors":["0","1","2","3","4","5","6","7","8","9"],"id":"20682"}'

In [169]:
help(factor_cmap)

Help on function factor_cmap in module bokeh.transform:

factor_cmap(field_name, palette, factors, start=0, end=None, nan_color='gray')
    Create a ``DataSpec`` dict that applies a client-side
    ``CategoricalColorMapper`` transformation to a ``ColumnDataSource``
    column.
    
    Args:
        field_name (str) : a field name to configure ``DataSpec`` with
    
        palette (seq[color]) : a list of colors to use for colormapping
    
        factors (seq) : a sequences of categorical factors corresponding to
            the palette
    
        start (int, optional) : a start slice index to apply when the column
            data has factors with multiple levels. (default: 0)
    
        end (int, optional) : an end slice index to apply when the column
            data has factors with multiple levels. (default: None)
    
        nan_color (color, optional) : a default color to use when mapping data
            from a column does not succeed (default: "gray")
    
    Returns:

In [185]:
factor_cmap("is_prime", palette=list(cmap.values()), factors=list(cmap.keys()))

{'field': 'is_prime', 'transform': CategoricalColorMapper(id='19211', ...)}

In [190]:
help(p.rect)

Help on method rect in module bokeh.plotting.figure:

rect(x, y, width, height, angle=0.0, dilate=False, *, angle_units='rad', fill_alpha=1.0, fill_color='gray', height_units='data', line_alpha=1.0, line_cap='butt', line_color='black', line_dash=[], line_dash_offset=0, line_join='bevel', line_width=1, name=None, tags=[], width_units='data', **kwargs) method of bokeh.plotting.figure.Figure instance
    Configure and add :class:`~bokeh.models.glyphs.Rect` glyphs to this Figure.
    
    Args:
        x (:class:`~bokeh.core.properties.NumberSpec` ):
            The x-coordinates of the centers of the rectangles.
    
        y (:class:`~bokeh.core.properties.NumberSpec` ):
            The y-coordinates of the centers of the rectangles.
    
        width (:class:`~bokeh.core.properties.DistanceSpec` ):
            The overall widths of the rectangles.
    
        height (:class:`~bokeh.core.properties.DistanceSpec` ):
            The overall heights of the rectangles.
    
        angle 